## Importa as Bibliotecas

- **Requests** para conectar ao Open Street Map.
- **pprint** para manuseio dos dados.

In [61]:
import requests
from pprint import pprint

In [62]:
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

- Solicita o Código Postal e transfere para URL.

In [63]:
# postcode = 'G42 9AY'
postcode = str(input('Código postal: ')).strip()
response = requests.get(f"{BASE_URL}&postalcode={postcode}")

Transforma a resposta no formato json, se falhar, retorna o conteúdo da resposta.

In [64]:
try:
    data = response.json()
    print(data)
except:
    print("Error from server: " + str(response.content))

[]


A partir da resposta em Json, é extraído a latitude e a longitude da localização informada.

In [65]:
latitude = data[0].get('lat')
longitude = data[0].get('lon')

IndexError: list index out of range

### Bibliotecas

- **Folium:** Para mostrar o mapa

### Código

- Localização recebe latitude e longitude;
- a variável m recebe a localização no mapa;
- um marcador é adicionado na localização informada.

In [ ]:
import folium

location = float(latitude), float(longitude)

m = folium.Map(location=location, width=800, height=400)
folium.Marker(location, popup='Sua localização!').add_to(m)

m

